In [ ]:
import requests
import json
import sys

In [ ]:
################################################################################
# Canvas login and course specific details
################################################################################

# Canvas API URL
API_URL = "https://canvas.wisc.edu/api/v1"

# Canvas API key - TODO enter your API key information here
# Please follow the steps for generating canvas API token
API_KEY = "HERE IS WHERE YOUR API KEY GOES"

# Canvas course number - TODO update this for your course
canvas_course = 322078
################################################################################


# Get the list of students and their canvas IDs to send the personal message
url = API_URL + "/courses/" + str(canvas_course) + "/students"
headers = {'Authorization': "Bearer " + API_KEY}
r = requests.get(url, headers = headers)
response_json = r.json()

In [ ]:
# Dict to keep track of wiscemail to canvas ID, name mapping
student_dict = {} # Key: wisc email; Value: dict containing canvas ID and student name
for student in response_json:
    wisc_email = student["login_id"].lower()
    name_parts = student["name"].split()
    first_name = name_parts[0]
    last_name = " ".join(name_parts[1:])
    if first_name == "Test":
        continue
    if wisc_email not in student_dict:
        student_dict[wisc_email] = {"canvas_id": student["id"], "first name": first_name}

# student_dict

In [ ]:
print("Total number of students:", len(student_dict))

In [ ]:
# Get user input for exam number
exam_name = input("Enter exam name: exam1 or exam2 or final: ")

In [ ]:
def read_json(path):
    with open(path, encoding = "utf-8") as f:
        return json.load(f)

In [ ]:
roster = read_json("{}_roster.json".format(exam_name))
                   
for student in roster:
    if student["enrolled"]:
        information = student[exam_name]
        email = student["email"]
        if email not in student_dict:
            print("PROBLEM: couldn't find student on canvas", student["name"])
        else:
            student_dict[email][exam_name] = information
            student_dict[email]["lecture"] = str(student["section"])

In [ ]:
# Uncomment to see student dictionary 
#student_dict

In [ ]:
# Assumes that one instructor teaches two sections
# If that changes, this cell's code requires an update 
# (Below conditional using first_lecture & second_lecture should be re-written)

first_lecture = input("Enter you first lecture number (enter 1 or 2 or 3 etc.,): ")
second_lecture = input("Enter you second lecture number (enter 1 or 2 or 3 etc.,): ")
lecture_list = ["1", "2", "3", "4", "5", "6"]

if first_lecture not in lecture_list or second_lecture not in lecture_list:
    print("Please enter proper input for lecture number (enter 1 or 2 or 3 etc.,). \
    Re-run this cell and provide proper input.")
else:
    # Send out the canvas messages
    for student_email in student_dict:
        first_name = student_dict[student_email]["first name"]
        information = student_dict[student_email][exam_name]
        section = student_dict[student_email]["lecture"]
        if section != first_lecture and section != second_lecture:
            # Skipping other lectures processing
            continue
        canvas_id = student_dict[student_email]["canvas_id"]
        title = exam_name.upper() + ' information'
        message = 'Dear {}, Here are your {} details: {}. - CS220 instructors'\
        .format(first_name, exam_name, information)
        payload = { 
                   'recipients' : [canvas_id], # canvas user_id
                   'subject' : title,
                   'body' : message,
                   'force_new' : True,
                   'context_code' : 'course_' + str(canvas_course) # if it's associated with a course
                  }
        
        # UNCOMMENT THE BELOW LINES TO SEND CANVAS MESSAGE
        #convo = requests.post(API_URL + "/conversations", headers = headers, data = payload)
        #convo.raise_for_status()
        print("Message sent to {}: {}".format(first_name, message))